# Lab 7: Human motion generation

## Advanced deep learning

## Setup

### Setup dataset


Please access this Google Drive folder: [link](https://drive.google.com/drive/folders/1V5yzlwBPSNVPj33SfDHnvMykXISh3CyB?usp=sharing) and create a shortcut in the root of your Google Drive `/content/drive/MyDrive/`.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/humanml3d-data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
caption_clip  caption_raw  checkpoints	humanml3d_test_split.txt  smplh  smpl_rifke


### Setup environment

Make sure you're running on a T4 GPU Colab instance; if not, activate it.

In [2]:
!nvidia-smi

Tue Mar  4 21:08:18 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch
torch.cuda.is_available()

True

Clone the lab repository.

In [3]:
# !git clone https://github.com/robincourant/lab5-CSC52087EP.git
# !git clone https://github.com/robincourant/lab-MotionDiT.git
!git clone https://github.com/siyuan-zou/Human_Motion_Diffusion.git

fatal: destination path 'Human_Motion_Diffusion' already exists and is not an empty directory.


In [4]:
# %cd lab5-CSC52087EP
%cd Human_Motion_Diffusion
!ln -s /content/drive/MyDrive/humanml3d-data ./ # Plug the dataset in the repo

/content/Human_Motion_Diffusion
ln: failed to create symbolic link './humanml3d-data': File exists


Install required libraries

In [5]:
!pip install hydra-core
!pip install pyrender
!pip install smplx
!pip install torchtyping
!pip install lightning
!pip install ema_pytorch

  Using cached pyrender-0.1.45-py3-none-any.whl.metadata (1.5 kB)
  Using cached freetype_py-2.5.1-py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (6.3 kB)
  Using cached pyglet-2.1.3-py3-none-any.whl.metadata (7.7 kB)
  Using cached PyOpenGL-3.1.0.zip (2.2 MB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.1/962.1 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.6/708.6 kB 42.8 MB/s eta 0:00:00
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=1745193 sha256=6e53d9eb2fcb0ddab63feebeef6b8d1a7e417724df53e4cc4e75bb74afe35421
  Stored in directory: /root/.cache/pip/wheels/2f/37/f5/f88cd3dddf75bc3ce608e44bf8a79078c408bf1f351a50818e
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
 

## Human motion dataset and representation

### HumanML3D dataset

#### Question 1:
*Overall, HumanML3D dataset consists of 14,616 motions and 44,970 descriptions composed by 5,371 distinct words. The total length of motions amounts to 28.59 hours. The average motion length is 7.1 seconds, while average description length is 12 words.*

### SMPL representation


#### Question 2:
betas: shape parameters that define body shape variations based on PCA coefficients. The number of betas used is defined by num_betas.

global_orient: the global orientation of the body in axis-angle representation, defining the overall rotation of the model.

body_pose: the relative rotations of the 23 body joints (excluding hands and face) in axis-angle representation.

v_template: the template mesh of the SMPL model, representing a neutral body shape.

shapedirs: shape blend shapes, defining how the body shape changes based on betas.

posedirs: pose blend shapes, defining how the pose affects the body surface.

J_regressor: a matrix that extracts joint locations from vertex positions.

parents: the kinematic tree defining joint hierarchy.

lbs_weights: skinning weights for Linear Blend Skinning (LBS), defining how each vertex is influenced by joint transformations.

#### Code 2
*Complete `visualize_smpl.py`*

In [7]:
!HYDRA_FULL_ERROR=1 PYTHONPATH=$(pwd) python src/visualize_smpl.py

import moviepy.editor
moviepy.editor.ipython_display("./smpl.mp4")

[2025-03-04 19:37:36,772][numexpr.utils][INFO] - NumExpr defaulting to 2 threads.
/content/Human_Motion_Diffusion/src/datasets/modalities/human_dataset.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related 

  if event.key is 'enter':



## Model architectures

### Config A: Incontext

#### Code 3
*Complete `src/models/modules/incontext.py`*

In [8]:
!PYTHONPATH=$(pwd) python src/models/modules/incontext.py

Test passed!


### Config B: AdaLN

#### Question 3:
\begin{equation}
\text{adaLN}(x, \gamma, \beta) = \gamma \cdot \frac{x - \mu}{\sigma} + \beta
\end{equation}

with $\mu$ being the mean of $x$ and $\sigma$ the standard deviation.

#### Code 4
*Complete `src/models/modules/adaln.py`*

In [9]:
!PYTHONPATH=$(pwd) python src/models/modules/adaln.py

Test passed!


### Config C: Cross attention

#### Question 4:
\begin{equation}
\text{CA}(x, c) = \text{softmax} \left( \frac{(x W_Q) (c W_K)^T}{\sqrt{d}} \right) (c W_V)
\end{equation}

#### Code 5
*Complete `src/models/modules/cross attention.py`*

In [10]:
!PYTHONPATH=$(pwd) python src/models/modules/cross_attention.py

Test passed!


#### Question 5:
*AdaLN applies conditioning through simple element-wise operations (scaling and shifting), making it computationally cheaper than cross-attention, which requires matrix multiplications and softmax computation.
Cross-attention involves a quadratic complexity $O(n^2d)$in the sequence length n, while AdaLN has only linear complexity $O(nd)$.*

## Diffusion framework

### DDPM

#### Code 6
*Complete `src/training/losses/ddpm.py`*

In [11]:
!PYTHONPATH=$(pwd) python src/training/losses/ddpm.py

Test passed!


#### Code 7
*Complete `src/training/sampler/ddpm.py`*

In [27]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/sampler@diffuser.test_sampler=ddpm seed=2

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_incontext.mp4")

Seed set to 2
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/Human_Motion_Diffusion/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use cas

### DDIM

#### Question 6:
*DDIM is faster than DDPM. The key difference is that DDIM is not stochastic.*

#### Code 8
*Complete `src/training/sampler/ddim.py`*

In [23]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/sampler@diffuser.test_sampler=ddim seed=2

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddim_incontext.mp4")

Seed set to 2
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/Human_Motion_Diffusion/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use cas

#### Question 7:
*The result of DDPM is more coherent with the caption as the result of DDIM also kicks with the right leg.*

## Result analysis

### Qualitative analysis

#### Code 9

In [28]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=incontext \
checkpoint_path=./humanml3d-data/checkpoints/incontext.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_incontext.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/Human_Motion_Diffusion/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use ca

In [29]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=adaln \
checkpoint_path=./humanml3d-data/checkpoints/adaln.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_adaln.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/Human_Motion_Diffusion/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use ca

In [30]:
!PYTHONPATH=$(pwd) python src/generate.py batch_size=1 diffuser/network=cross_attention \
checkpoint_path=./humanml3d-data/checkpoints/cross_attention.ckpt

import moviepy.editor
moviepy.editor.ipython_display("./generation_ddpm_cross_attention.mp4")

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
/content/Human_Motion_Diffusion/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use ca

#### Question 8
*The result of incontext and cross attention is correct while adaln cannot generate correct result. This means that attention helps in understanding the caption. And the incontext generates the best result as we learn the relationships between caption and human features other than only understanding the captions.*

### Quantitative analysis

#### Question 9:
*The main assumption when computing the Fréchet Distance is that both the reference and generated feature distributions follow a multivariate Gaussian distribution.*

#### Code 10
*Complete src/metrics/frechet.py*

#### Code 11
*Complete src/metrics/similarity.py*

#### Code 12

#### Bonus 1:
*Answer here...*

In [ ]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=incontext \
checkpoint_path=./humanml3d-data/checkpoints/incontext.ckpt

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_predict_batches=1)` was configured so 1 batch will be used.
[2025-03-04 21:11:17,433][OpenGL.acceleratesupport][INFO] - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
/content/Human_Motion_Diffusion/src/metrics/modules/tmr.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly a

In [ ]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=adaln \
checkpoint_path=./humanml3d-data/checkpoints/adaln.ckpt

In [ ]:
!PYTHONPATH=$(pwd) python src/evaluate.py diffuser/network=cross_attention \
checkpoint_path=./humanml3d-data/checkpoints/cross_attention.ckpt

#### Question 10:
*Answer here...*

#### Bonus 2:
*Answer here...*